In [ ]:
import sys

sys.path.append("..")
from core.DataLoader import (
    DataPreprocessor,
    DataConfig,
    LoadConfig,
    get_load_config_from_yaml,
)
import numpy as np
from importlib import reload
import matplotlib.pyplot as plt
import yaml
import core.assignment_models as Models
import core
import keras


PLOTS_DIR = f"plots/assignment_transformer/"
MODEL_DIR = f"models/assignment_transformer/"
CONFIG_PATH = "../config/workspace_config.yaml"

import os

if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)


load_config = get_load_config_from_yaml(CONFIG_PATH)

DataProcessor = DataPreprocessor(load_config)


with open(CONFIG_PATH, "r") as file:
    data_configs = yaml.safe_load(file)
plt.rcParams.update({"font.size": 14})

data_config = DataProcessor.load_from_npz(
    data_configs["data_path"]["nominal"], max_events=1000_000
)
X_train, y_train, X_val, y_val = DataProcessor.split_data(test_size=0.1)
del DataProcessor

2025-11-28 09:46:26.005042: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764319586.027086 2495698 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764319586.034361 2495698 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764319586.052008 2495698 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764319586.052030 2495698 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764319586.052032 2495698 computation_placer.cc:177] computation placer alr

In [2]:
reload(Models)
reload(core)
TransformerMatcher = Models.FeatureConcatTransformer(
    data_config, name=r"Transformer + $\nu^2$-Flows"
)

# TransformerMatcher.load_model(f"{MODEL_DIR}/model.keras")
TransformerMatcher.build_model(
    hidden_dim=64,
    num_layers=6,
    num_heads=8,
    dropout_rate=0.1,
)

TransformerMatcher.adapt_normalization_layers(X_train)

TransformerMatcher.compile_model(
    loss={
        "assignment": core.utils.AssignmentLoss(),
    },
    optimizer=keras.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-4),
    metrics={
        "assignment": [core.utils.AssignmentAccuracy()],
    },
)

FeatureConcatTransformer is designed for classification tasks; regression targets will be ignored.


I0000 00:00:1764319772.188923 2495698 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15511 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:65:00.0, compute capability: 6.0


Building model without regression output.


In [3]:
TransformerMatcher.train_model(
    epochs=100,
    X_train=X_train,
    y_train=y_train,
    sample_weights=core.utils.compute_sample_weights(X_train, y_train),
    batch_size=1024,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.5,
            patience=5,
            min_lr=1e-6,
            verbose=1,
        ),
    ],
)

Epoch 1/100


I0000 00:00:1764319794.260921 2500211 service.cc:152] XLA service 0x7f08cc00f8e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1764319794.260945 2500211 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2025-11-28 09:49:54.898930: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-11-28 09:49:55.582640: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert
I0000 00:00:1764319797.366574 2500211 cuda_dnn.cc:529] Loaded cuDNN version 91500


  7/703 ━━━━━━━━━━━━━━━━━━━━ 12s 17ms/step - accuracy: 0.1267 - loss: 0.1910 

I0000 00:00:1764319811.332941 2500211 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


700/703 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.2024 - loss: 0.1603

2025-11-28 09:50:23.112637: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert
2025-11-28 09:50:36.271510: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'input_multiply_reduce_select_fusion', 12 bytes spill stores, 12 bytes spill loads



703/703 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.2027 - loss: 0.1602

2025-11-28 09:50:37.815182: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert
2025-11-28 09:50:39.796531: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert


703/703 ━━━━━━━━━━━━━━━━━━━━ 62s 42ms/step - accuracy: 0.2027 - loss: 0.1602 - val_accuracy: 0.4430 - val_loss: 0.1256 - learning_rate: 1.0000e-04
Epoch 2/100
703/703 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - accuracy: 0.4170 - loss: 0.1321 - val_accuracy: 0.5801 - val_loss: 0.1052 - learning_rate: 1.0000e-04
Epoch 3/100
703/703 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - accuracy: 0.5224 - loss: 0.1163 - val_accuracy: 0.6400 - val_loss: 0.0954 - learning_rate: 1.0000e-04
Epoch 4/100
703/703 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - accuracy: 0.5764 - loss: 0.1084 - val_accuracy: 0.6693 - val_loss: 0.0909 - learning_rate: 1.0000e-04
Epoch 5/100
703/703 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - accuracy: 0.6103 - loss: 0.1036 - val_accuracy: 0.6840 - val_loss: 0.0870 - learning_rate: 1.0000e-04
Epoch 6/100
703/703 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - accuracy: 0.6337 - loss: 0.0999 - val_accuracy: 0.7002 - val_loss: 0.0847 - learning_rate: 1.0000e-04
Epoch 7/100
703/703 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - a

In [4]:
TransformerMatcher.save_model(f"{MODEL_DIR}/model.keras")

Model saved to models/assignment_transformer//model.keras
